In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import sys
sys.path.append("./CLIP")
import clip
import os
import torch
from PIL import Image
from IPython.display import Image as ImageDisplay
from utils import load_pickle
import faiss_utils
from faiss_utils import KNearestFaiss
import numpy as np
from matplotlib.image import imread
from matplotlib.pyplot import figure, imshow, axis
from jupyter_utils import plot_time_jupyter, plot_scores_jupyter

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
def display_top_query_from_meta(metadata_list, fill=True, score_list=[]):
    list_of_files = [meta.get_path() for meta in metadata_list]
    show_images(list_of_files, fill=fill, score_list=score_list)
        

def show_images(list_of_files, fill=True, score_list=[]):
    block_size = int(len(list_of_files)/20)
    size = (20+20*block_size,20+90*block_size)
    print(size)
    fig = figure(figsize=size, tight_layout=True)
    number_of_files = len(list_of_files)
    for i in range(number_of_files):
        a=fig.add_subplot(number_of_files,3,i+1)
        image = imread(list_of_files[i])
#         imshow(image,cmap='Greys_r')
        if fill:
            imshow(image,aspect="auto")
        else:
            imshow(image)
        if len(score_list) > 0:
            title=score_list[i]
            plt.title(title, fontsize=30); 
        axis('off')

def display_query_dict_random(query_dict, class_start=0, class_end=9, fill=False):
    query_names = list(query_dict.keys())[class_start:class_end+1]
    image_dict = {}
    rank_list=list(range(0, 999, 40))
    for query_name in query_names:
        list_of_files, list_of_scores, rank_list = select_from_meta(query_dict[query_name]['metadata'], query_dict[query_name]['D'], rank_list=rank_list)
        image_dict[query_name] = list_of_files, list_of_scores
    show_image_dict(image_dict, fill=fill)

def display_query_dict(query_dict, class_start=0, class_end=9, fill=False):
    query_names = list(query_dict.keys())[class_start:class_end+1]
    image_dict = {}
    for query_name in query_names:
        list_of_files, list_of_scores, rank_list = select_from_meta(query_dict[query_name]['metadata'], query_dict[query_name]['D'])
        image_dict[query_name] = list_of_files, list_of_scores
    show_image_dict(image_dict, fill=fill)
        
    
def select_from_meta(metadata_list, score_list, rank_list=[0, 199, 399, 599, 799, 999]):
    list_of_files = [metadata_list[rank].get_path() for rank in rank_list]
    list_of_scores = [score_list[rank] for rank in rank_list]
    return list_of_files, list_of_scores, rank_list

def show_image_dict(image_dict, fill=True):
    block_size = 2
    size = (20+20*block_size,20+90*block_size)
    size = (20+20*block_size,20+420*block_size)
    print(size)
    fig = figure(figsize=size, tight_layout=True)
    number_of_queries = len(image_dict.keys())
    list_of_files = []
    list_of_scores = []
    list_of_queries = []
    for query in image_dict.keys():
        files, scores= image_dict[query]
        list_of_files += files
        list_of_queries += [query for _ in files]
        list_of_scores += scores
        length_per_query = len(files)
    number_of_files = len(list_of_files)
    
    for i in range(number_of_files):
        a=fig.add_subplot(number_of_files,int(length_per_query/5),i+1)
        if i % length_per_query == 0:
            a.set_ylabel(list_of_queries[i],  fontsize=28)

        image = imread(list_of_files[i])
#         imshow(image,cmap='Greys_r')
        if fill:
            imshow(image,aspect="auto")
        else:
            imshow(image)
        if len(list_of_scores) > 0:
            title=list_of_scores[i]
            plt.title(title, fontsize=30); 
        axis('on')

In [5]:
folder_path = "/scratch/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_feb_28/"
# model_name = "ViT-B/32"
model_name = "RN50"


# feature_name = faiss_utils.get_feature_name(folder_path, model_name)
# normalize_features = load_pickle(feature_name)
# normalize_features.dtype == np.float32
k_near_faiss = KNearestFaiss(folder_path, model_name)

Loading from folder /scratch/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_31/all_folders.pickle
Size of dataset is 2740000
Need to use brute force search without building an index


In [6]:

# image_path = "/compute/autobot-0-19/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_25/1/images/5411319415.jpg"
image_path = '/scratch/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_31/216/images/2375750164.jpg'
normalize_image_feature = k_near_faiss.get_normalized_image_feature(image_path)

# D, meta_list = k_near_faiss.k_nearest_meta(normalize_image_feature, k=3)


In [7]:
D, meta_list = k_near_faiss.k_nearest_meta(normalize_image_feature, k=3)

RuntimeError: Error in virtual void* faiss::gpu::StandardGpuResourcesImpl::allocMemory(const faiss::gpu::AllocRequest&) at /__w/faiss-wheels/faiss-wheels/faiss/faiss/gpu/StandardGpuResources.cpp:410: Error: 'err == cudaSuccess' failed: Failed to cudaMalloc 1610612736 bytes on device 0 (error 2 out of memory
Outstanding allocations:


In [ ]:
ImageDisplay(image_path)

In [ ]:
display_top_query_from_meta(meta_list)
meta_list[0].get_path()

In [ ]:
import time
def show_top_query_images(query, start_idx=0, end_idx=40, fill=True):
    start = time.time()
    normalize_text_feature = k_near_faiss.get_normalized_text_feature(query)
    end_feature = time.time()
    score_list, meta_list = k_near_faiss.k_nearest_meta(normalize_text_feature, k=end_idx)
    end_search = time.time()
    print(f"{end_feature-start:.4f} for computing features. {end_search-end_feature} for computing KNN.")
    display_top_query_from_meta(meta_list[start_idx:end_idx], fill=fill, score_list=score_list[start_idx:end_idx])

def grab_top_query_images(query, start_idx=0, end_idx=40):
    start = time.time()
    normalize_text_feature = k_near_faiss.get_normalized_text_feature(query)
    end_feature = time.time()
    score_list, meta_list = k_near_faiss.k_nearest_meta(normalize_text_feature, k=end_idx)
    end_search = time.time()
    print(f"{end_feature-start:.4f} for computing features. {end_search-end_feature} for computing KNN.")
    print(f"length is {len(meta_list)} and {len(meta_list[start_idx:end_idx])}")
    return score_list[start_idx:end_idx], meta_list[start_idx:end_idx]

def grab_top_query_images_difference(query, diff_queries, start_idx=0, end_idx=40, lmb=0.25):
    start = time.time()
    normalize_text_feature = k_near_faiss.get_text_difference_feature(query, diff_queries=diff_queries, lmb=lmb)
    end_feature = time.time()
    score_list, meta_list = k_near_faiss.k_nearest_meta(normalize_text_feature, k=end_idx)
    end_search = time.time()
    print(f"{end_feature-start:.4f} for computing features. {end_search-end_feature} for computing KNN.")
    print(f"length is {len(meta_list)} and {len(meta_list[start_idx:end_idx])}")
    return score_list[start_idx:end_idx], meta_list[start_idx:end_idx]
#     print(meta_lists[0][7].get_path())

In [ ]:

fruit_apple = 'a photo of apple, a type of fruit'
phone_apple = 'a photo of apple, a type of phone'
logo_apple = 'a photo of apple, a type of logo'
expensive_apple = 'a photo of expensive apple'
cheap_apple = 'a photo of cheap apple'
just_apple = 'a photo of apple'

just_an_apple = 'a photo of an apple'
just_apples = 'a photo of apples'
just_apple = 'a photo of apple'
apple_tree = 'a photo of apple tree'

no_apple = 'no apple'

old_apple = 'a photo of apple, which is old'
new_apple = 'a photo of apple, which is new'
fresh_apple = 'a photo of apple, which is fresh'
rotten_apple = 'rotten apple'
pretty_apple = 'a photo of apple, which is pretty'
ugly_apple = 'a photo of apple, which is ugly'
broken_apple = 'broken apple'
no_function_apple = 'not functioning apple'


old_car = "a photo of a car, which is old"
a1980_car = "a photo of a car in 1980s"
n2004_car = "a photo of a car in 2004"
n2005_car = "a photo of a car in 2005"
n2006_car = "a photo of a car in 2006"
n2007_car = "a photo of a car in 2007"
n2008_car = "a photo of a car in 2008"
n2009_car = "a photo of a car in 2009"
n2010_car = "a photo of a car in 2010"
n2011_car = "a photo of a car in 2011"
n2012_car = "a photo of a car in 2012"
n2013_car = "a photo of a car in 2013"
n2014_car = "a photo of a car in 2014"
a2010_car = "a photo of a car in 2010s"
new_car = "a photo of a car, which is new"
broken_car = "a photo of a car, which is broken"
just_car = "a photo of a car"

old_bike = "a photo of a bike, which is old"
new_bike = "a photo of a bike, which is new"
broken_bike = "a photo of a bike, which is broken"
just_bike = "a photo of a bike"

old_motorcycle = "a photo of a motorcycle, which is old"
new_motorcycle = "a photo of a motorcycle, which is new"
broken_motorcycle = "a photo of a motorcycle, which is broken"
just_motorcycle = "a photo of a motorcycle"
just_motorcycles = "a photo of motorcycles"

old_motorbike = "a photo of a motorbike, which is old"
new_motorbike = "a photo of a motorbike, which is new"
broken_motorbike = "a photo of a motorbike, which is broken"
just_motorbike = "a photo of a motorbike"
only_motorbike = "only motorbike"
motorbike_and_car = "motorbike and car"

no_car_motorbike = "a photo of a motorbike, no car"

old_jet = "a photo of a jet, which is old"
new_jet = "a photo of a jet, which is new"
broken_jet = "a photo of a jet, which is broken"

old_shirt = "a photo of a t-shirt, which is old"
new_shirt = "a photo of a t-shirt, which is new"

n2004 = 'a photo in 2004'
n2014 = 'a photo in 2014'
n1980 = 'a photo in 1980'
n2020 = 'a photo in 2020'


mouse_animal = 'a photo of mouse, a type of animal'
mouse_accessory = 'a photo of mouse, a type of accessory'
just_mouse = 'a photo of mouse'
just_star = 'a photo of star'

obama = 'obama'
trump = 'trump'
president = 'a photo of president'
curr_president = 'a photo of current president'
vice_president = 'a photo of vice president'
presidential_debate = "presidential debate"

In [ ]:
motor_bike_no_car_scores, motor_bike_no_car_lst = grab_top_query_images_difference("A photo of a motorcycle", diff_queries=['A photo of a car'],\
                                                                     start_idx=0, end_idx=10000, lmb=1.)


In [ ]:
start=500
end=start+50
display_top_query_from_meta(motor_bike_no_car_lst[start:end], fill=False, score_list=motor_bike_no_car_scores[start:end])
print(motor_bike_no_car_scores[end])
print(motor_bike_no_car_lst[8].get_path())

In [ ]:
motor_bike_with_car_scores, motor_bike_with_car_lst = grab_top_query_images_difference("A photo of a motorcycle", diff_queries=['A photo of a car'],\
                                                                     start_idx=0, end_idx=10000, lmb=-1.)


In [ ]:
start=100
end=start+50
display_top_query_from_meta(motor_bike_with_car_lst[start:end], fill=False, score_list=motor_bike_with_car_scores[start:end])

In [ ]:
# show_top_query_images("A photo of motorbike without car", start_idx=0, end_idx=50, fill=False)
motor_bike_scores, motor_bike_lst = grab_top_query_images("A photo of a motorcycle", start_idx=0, end_idx=10000)

In [ ]:
plot_scores_jupyter(motor_bike_scores)

In [ ]:
def plot_scores_jupyter(score_list, plot_mean=False):
    min_count = None
    max_count = None
    avg_count = 0
    for s in score_list:
        if not min_count or s < min_count:
            min_count = s
        if not max_count or s > max_count:
            max_count = s
        avg_count += s
    avg_count = avg_count / len(score_list)

    bins = np.linspace(min_count, max_count, 15)
    plt.figure(figsize=(8,8))
    if plot_mean:
        plt.axhline(y=avg_count, label=f"Mean scores of Samples {avg_count}", linestyle='--', color='black')
    plt.hist(score_list, bins, alpha=0.5, label='Number of samples')
    plt.legend(loc='upper right')
    plt.tight_layout()

In [ ]:
start=100
end=start+50
display_top_query_from_meta(motor_bike_lst[start:end], fill=False, score_list=motor_bike_scores[start:end])
print(motor_bike_scores[end])
print(motor_bike_lst[8].get_path())

In [ ]:
motorcycle_with_car_path = "/scratch/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_31/33/images/3878798703.jpg"
motorcycle_without_car_path = motor_bike_lst[8].get_path()
ImageDisplay(motorcycle_with_car_path)


In [ ]:
normalize_image_feature = k_near_faiss.get_normalized_image_feature(motorcycle_with_car_path)
D, meta_list = k_near_faiss.k_nearest_meta(normalize_image_feature, k=20)

In [ ]:
idx = 10
display_top_query_from_meta(meta_list, fill=False)
p = meta_list[idx].get_path()
motor_score = k_near_faiss.get_clip_score(p, "A photo of a motorcycle")
car_score = k_near_faiss.get_clip_score(p, "A photo of a car")
print(f"motorcycle score: {motor_score}")
print(f"car score: {car_score}")
# ImageDisplay(p)

In [ ]:
motor_score = k_near_faiss.get_clip_score(motorcycle_with_car_path, "A photo of a motorcycle")
car_score = k_near_faiss.get_clip_score(motorcycle_with_car_path, "A photo of a car")
print(f"motorcycle score: {motor_score:.4f}")
print(f"car score:        {car_score:.4f}")
lmb = 1.0
diff_clip_score = k_near_faiss.get_clip_score(motorcycle_with_car_path, query="A photo of a motorcycle",\
                            diff_queries=["A photo of a car"],\
                           lmb=lmb)
print(f"Score difference: {diff_clip_score:.4f}")
ImageDisplay(motorcycle_with_car_path)

In [ ]:
motor_score = k_near_faiss.get_clip_score(motorcycle_without_car_path, "A photo of a motorcycle")
car_score = k_near_faiss.get_clip_score(motorcycle_without_car_path, "A photo of a car")
print(f"motorcycle score: {motor_score:.4f}")
print(f"car score:        {car_score:.4f}")
lmb = 1.0
diff_clip_score = k_near_faiss.get_clip_score(motorcycle_without_car_path, query="A photo of a motorcycle",\
                            diff_queries=["A photo of a car"],\
                           lmb=lmb)
print(f"Score difference: {diff_clip_score:.4f}")
ImageDisplay(motorcycle_without_car_path)

In [ ]:
apple_scores, apple_lst = grab_top_query_images("A photo of a apple", start_idx=0, end_idx=10000)

In [ ]:
# display_top_query_from_meta(apple_lst[:50], fill=False, score_list=apple_scores[:50])
start=300
end=start+50
display_top_query_from_meta(apple_lst[start:end], fill=False, score_list=apple_scores[start:end])
print(apple_scores[end])

In [ ]:
plot_scores_jupyter(apple_scores)

In [ ]:
bike_scores, bike_lst = grab_top_query_images("A photo of a bike", start_idx=0, end_idx=10000)

In [ ]:
start=600
end=start+50
display_top_query_from_meta(bike_lst[start:end], fill=False, score_list=bike_scores[start:end])
print(bike_scores[end])
# print(bike_lst[8].get_path())

In [ ]:
bike_no_motor_bike_scores, bike_no_motor_bike_lst = grab_top_query_images_difference("A photo of a bike", diff_queries=['A photo of a motorcycle'],\
                                                                     start_idx=0, end_idx=10000, lmb=1.)



In [ ]:
start=500
end=start+50
display_top_query_from_meta(bike_no_motor_bike_lst[start:end], fill=False, score_list=bike_no_motor_bike_scores[start:end])
print(bike_no_motor_bike_scores[end])

In [ ]:
bike_no_motor_bike_no_car_scores, bike_no_motor_bike_no_car_lst = grab_top_query_images_difference("A photo of a bike", \
                                                                                                   diff_queries=['A photo of a motorcycle', 'A photo of a car'],\
                                                                     start_idx=0, end_idx=10000, lmb=1.)




In [ ]:
start=600
end=start+50
display_top_query_from_meta(bike_no_motor_bike_no_car_lst[start:end], fill=False, score_list=bike_no_motor_bike_no_car_scores[start:end])
print(bike_no_motor_bike_no_car_scores[end])

In [ ]:
car_scores, car_lst = grab_top_query_images("A photo of car", start_idx=0, end_idx=4000)

In [ ]:
# meta_lst = grab_top_query_images(phone_apple, start_idx=0, end_idx=2049)
# plot_time_jupyter(meta_lst)

In [ ]:
lst = grab_top_query_images(fruit_apple, start_idx=0, end_idx=500)
plot_time_jupyter(lst)

In [ ]:
lst = grab_top_query_images(fruit_apple, start_idx=0, end_idx=5000)
plot_time_jupyter(lst)

In [ ]:
# ImageDisplay("/scratch/zhiqiu/yfcc100m_all/images_minbyte_10_valid_uploaded_date_jan_25/37/images/11052978605.png")